# Common Size Income Statement

**Purpose**: The purpose of this notebook is to obtain the Income Statement table from a Company filing (10-Q, 10-K) using the filing url. Edgar nicely provides just the table, which can be easily extracted using Pandas for further analysis. We want to convert the statement to a common size statement by dividing each year by Gross Revenue. As a result, we can see each caption relative to our divisor.

## Imports

In [1]:
from Helpers.CompanyFiling import Filing
import re

## Set Up

Get the URL for the edgar filing. This is the "index-htm" page edgar directs you to when you click on a quarterly/annual report. We sould convert the url to redirect us to the folder structure of the filing package. Once inside, the folder has a file called the "FilingSummary.xml" which describes each item in the folder. We will scrape the filing summary to identify the file the statement is stored in. 

In [2]:
edgar_filing_url = "https://www.sec.gov/Archives/edgar/data/1617640/000161764021000012/0001617640-21-000012-index.html"
base_url, filing_summary_url = Filing.convert_filing_to_folder(edgar_filing_url=edgar_filing_url)
slugs = Filing.retrieve_face_report_slugs(filing_summary_url)
income_statement_url = base_url + "/" + slugs["IS"]

## Clean

In order to analyze the statement, we first use pandas to read the table form the URL we discovered above. We clean the dataframe by renaming the first column header,  converting the multi-index columns to be a single index, and converting the data types to be numeric. Next, we remove rows where all numeric columns are all missing.

Lastly, we return the common size statement by dividng by the Total Assets row.

In [3]:
is_df = Filing.get_clean_table(income_statement_url)

# ignore rows where the caption contains square brackets, typically disclosures. We ignore these for the sake of our quick analysis. Footnotes on the face will be evaluated
# when reading through the annual report
is_df = is_df.query("~Captions.fillna('').str.contains('\[')") 
is_df = is_df.query("Captions.notnull()")

# drop columns where 90% is missing
is_df = is_df.dropna(thresh=len(is_df)*.1, axis=1)
is_df = is_df.pipe(Filing.clean_dataframe)
is_df = is_df.pipe(Filing.remove_rows_of_zeros)

is_df = is_df.drop_duplicates(subset=["Captions"], keep="first")

is_df = Filing.common_size_financial_statement(is_df, divisor=is_df.select_dtypes("number").iloc[0])

Unable to clean column - Captions.
Unable to clean column - Dec. 31, 2020.
Unable to clean column - Dec. 31, 2019.
Unable to clean column - Captions.


## Visualize

We use pandas styler objects to add padding, add background colors, and format our numbers.

In [4]:
is_df.style\
    .background_gradient(cmap='Blues', axis=0,)\
    .set_properties(**{'padding': '10px'})\
    .hide_index()\
    .format({col: '{:,.2%}'.format for col in is_df.select_dtypes("number").columns })\


Captions,"Dec. 31, 2020","Dec. 31, 2019","Dec. 31, 2018"
Revenue,100.00%,100.00%,100.00%
Cost of revenue,52.59%,52.21%,11.52%
Sales and marketing,20.15%,26.04%,41.44%
Technology and development,15.51%,17.40%,30.81%
General and administrative,10.69%,13.35%,19.66%
Impairment costs,2.30%,0.00%,5.92%
Acquisition-related costs,0.00%,0.00%,0.17%
Integration costs,0.00%,0.02%,0.15%
Total costs and expenses,101.24%,109.02%,109.67%
Loss from operations,-1.24%,-9.02%,-9.67%
